In [4]:
using Printf

In [8]:
using SeisProcessing

In [9]:
d = SeisHypEvents(apex=[100, 200, -300], f0=[30, 20, 15]);

In [13]:
d=ones(10,10);


d[:,1]

10-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [1]:
function adjoint_hyp(d::Array{Float64,2}, indx::Array{Int,2}, dt::Float64,
                     nt::Int,h::Array{Float64,1}, tau::Array{Float64,1},
                     vel::Array{Float64,1}, apex::Array{Float64,1},
                     fbp::Array{Float64,1})

    nh = length(h)
    d  = bandpass(d, dt, fbp=fbp)
    nm = size(indx,1)
    m  = zeros(Float64,nm)
    @inbounds for im = 1:nm, ih = 1:nh
        itau  = indx[im,1] #tau index
        ivel  = indx[im,2] #velocity index
        iapex = indx[im,3] # apex index
        aux = (h[ih] - apex[iapex])/vel[ivel];
        tt  = sqrt(tau[itau]^2 + aux^2)
        it  = tt/dt + 1 
        it1 = floor(Int,it)
        it2 = it1 + 1
        a = it - it1
        if it2 <= nt
            m[im] += (1-a)*d[it1,ih] + a*d[it2,ih]
        end
    end
    return m
    
end


adjoint_hyp (generic function with 1 method)

In [2]:
function forward_hyp(m::Array{Float64,1}, indx::Array{Int,2}, dt::Float64,
                     nt::Int, h::Array{Float64,1}, tau::Array{Float64,1},
                     vel::Array{Float64,1}, apex::Array{Float64,1},
                     fbp::Array{Float64,1})
    
    nh = length(h)
    d  = zeros(Float64, nt, nh)
    nm = size(indx,1)
    @inbounds for im = 1:nm, ih = 1:nh
        itau  = indx[im,1]
        ivel  = indx[im,2]
        iapex = indx[im,3]
        aux = (h[ih] - apex[iapex])/vel[ivel]
        tt  = sqrt(tau[itau]^2 + aux^2)
        it  = tt/dt + 1
        it1 = floor(Int,it)
        it2 = it1 + 1
        a = it - it1
        if it2 <= nt
            d[it1,ih] += (1-a)*m[im]
            d[it2,ih] += a*m[im]
        end
    end
    d = bandpass(d, dt, fbp=fbp)
    return d
    
end


forward_hyp (generic function with 1 method)

In [7]:
function fista(d::Array{Float64,2}, adjoint_op::Function, forward_op::Function,
               indx::Array{Int,2}, dt::Float64, nt::Int, h::Array{Float64,1},
               tau::Array{Float64,1}, vel::Array{Float64,1},
               apex::Array{Float64,1}, fbp::Array{Float64,1}, alpha::Float64;
               Nit::Int=500, res::Float64=1e4, mu::Float64=0.1, say::Bool=true)

    println("")
    println(" ==================================================")
    println(" Fast Iterative Soft Thresholding Algorithm (FISTA)")
    println(" ==================================================")

    param = [dt, nt, h, tau, vel, apex, fbp]
    J = zeros(Float64, Nit+1)
    misfit = vecnorm(d)^2
    J[1] = misfit
    say ? @printf("\n Iteration: %3d, Misfit = %0.4f\n", 0, misfit) : nothing
    T = mu / (2*alpha)

    # Initialize
    m = zeros(adjoint_op(d, indx, param...))
    yk = copy(m)
    t = 1.0
    k = 0
    cost0 = misfit
    err = 1.0
    
    while k < Nit && misfit > res && err > 1e-4 

        # Update model m with FISTA
        k = k + 1 
        mk = copy(m)
        df = d - forward_op(yk, indx, param...)
        m  = adjoint_op(df, indx, param...)
        m  = thresh(yk + m/lambda, T, "soft")
        
        # Update cost function
        df = d - forward_op(m, indx, param...)
        misfit = vecnorm(df)^2
        cost = misfit + mu*sum(abs(m))
        J[k+1] = cost
        say ? @printf(" Iteration: %3d, Misfit = %0.4f, Cost = %0.4f\n", k,
                      misfit, cost) : nothing

        # Update t and yk for next iteration
        tk = t
        t  = 0.5 * (1.0 + sqrt(1.0 + 4.0*tk^2))
        yk = m + (tk-1.0)/t * (m-mk)
        err = abs(cost0-cost)/cost
 
    end

    return m, J

end


function power_method(m::Array{Float64,1}, adjoint_op::Function,
                      forward_op::Function, indx::Array{Int,2}, dt::Float64,
                      nt::Int, h::Array{Float64,1}, tau::Array{Float64,1},
                      vel::Array{Float64,1}, apex::Array{Float64,1},
                      fbp::Array{Float64,1}; Nit::Int=200, say::Bool=true)

    param = [dt, nt, h, tau, vel, apex, fbp]
    lambda = 1e-8
    k = 0
    err = 1.e4
    while k < Nit && err > 1e-8
        k = k + 1
        lambda0 = copy(lambda)
        d = forward_op(m, indx, param...)
        m1 = adjoint_op(d, indx, param...)
        lambda = norm(m)
        m = m1 / lambda
        err = abs((lambda0 - lambda)/lambda)
        say ? @printf(" Iteration: %3d, maximum eig: %0.4f, rel. dif.: %0.8f\n",
                      k, lambda, err) : nothing
    end
    return lambda

end


function thresh(x::Array{Float64,1}, t::Float64, kind::String)

    if kind=="soft"
        tmp = abs(x) - t
        tmp = (tmp + abs(tmp)) / 2
        x   = sign(x) .* tmp
    elseif kind =="hard"
        x   = x.*(abs(x).>t);
    else
        error("Wrong thresholding kind")
    end
    return x

end




thresh (generic function with 1 method)